[문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough 
- itemgetter()

세션ID 설정하고, 대화를 진행합니다. (multi-turn conversation) 
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.

In [1]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv


## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'
with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)


In [8]:
## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    # model='gpt-4.1-nano',
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    max_tokens=512,
)

from langchain_core.messages import trim_messages

## 6. trim 설정
trimmer = trim_messages(
   max_tokens=65, # 최근 토큰 수 제한
   strategy="last", # 최근 메시지부터 시작해서 자르기
   token_counter=llm, # 모델 기준으로 토큰 수 계산
   include_system=True, # 시스템 메시지 포함
   allow_partial=False, #메시지 자르는걸 제한
   start_on='human', #human 메세지부터 자르기기
)

from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

## 6. chain 구성
chain = (
        RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
        | prompt 
        | llm
)

## 7. 세션별 대화 이력(히스토리)을 저장할 임시 메모리 저장소
store = {}

## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

## 11. 실행

while True:
  
  query = input('육아휴직과 관련된 질문하세요. 질문이 없다면 S를 입력 >>> ')

  if query.strip().upper() == "S":
    print('대화를 종료합니다.')
    break

  
  with_message_history.invoke(
    {'law': law, 'query': query},
    config={'configurable': {'session_id': 'abc'}}
    )

IndexError: list index out of range

In [3]:
store

{'abc': InMemoryChatMessageHistory(messages=[])}

In [15]:
for message in get_session_history('abc').messages:
  print(f'[{message.type.upper()}]:{message.content}\n')

[HUMAN]:육아휴직 기간은 얼마인가요?

[AI]:육아휴직의 기간은 1년 이내입니다(제19조(육아휴직) ②항). 다만, 특별한 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있는데, 이는 다음과 같은 경우에 해당합니다:
- 같은 자녀를 대상으로 부모가 각각 3개월 이상 사용한 경우
- 「한부모가족지원법」 제4조제1호의 부 또는 모
- 고용노동부령으로 정하는 장애아동의 부 또는 모(제19조(육아휴직) ②항).

